In [1]:
import PIL
import pandas as pd
import numpy as np
import os
import matplotlib
import json
import math
from random import randint
from IPython.display import Image 

In [4]:
resultPath = "../model_tests/Move Classifier/"

results = []
for path in os.listdir(resultPath):
    if not '.DS_Store' in path:
        try:
            
            args = dict(zip(['feature', 'balance', 'batchsize', 'IW', 'IH', 'subset'], path.split("-")))
            args['balance'] = 1 if args['balance'][3:]=="True" else 0
            args['batchsize'] = int(args['batchsize'][2:])
            args['IW'] = int(args['IW'][2:])
            args['IH'] = int(args['IH'][2:])
            args['subset'] = float(args['subset'][3:])
            
            with open(resultPath + path + "/time.json") as json_file:
                time = json.load(json_file)
                args.update(time)
            
            df = pd.read_csv(resultPath + path + "/predictions.csv")
            predictions = {"TPImages": [], "TNImages": [], "FPImages": [], "FNImages": [],
                          "TPPredictions": [], "TNPredictions": [], "FPPredictions": [], "FNPredictions": []}
            SSE = 0
            for index, row in df.iterrows():
                if row['results'] == 'TP':
                    predictions['TPImages'].append(row['image'])
                    predictions['TPPredictions'].append(row['prediction'])
                elif row['results'] == 'TN':
                    predictions['TNImages'].append(row['image'])
                    predictions['TNPredictions'].append(row['prediction'])
                elif row['results'] == 'FP':
                    predictions['FPImages'].append(row['image'])
                    predictions['FPPredictions'].append(row['prediction'])
                elif row['results'] == 'FN':
                    predictions['FNImages'].append(row['image'])
                    predictions['FNPredictions'].append(row['prediction'])
                else:
                    "This should never happen."
                    
                SSE += (row['prediction'] - row['true'])**2
            
            print(len(predictions['TPImages']), len(predictions['TNImages']), len(predictions['FPImages']), len(predictions['FNImages']))
            
            recall = len(predictions['TPImages'])/(len(predictions['TPImages'])+len(predictions['FNImages']))      
            precision = len(predictions['TPImages'])/(len(predictions['TPImages'])+len(predictions['FPImages']))
            predictions.update({"RMSE": math.sqrt(SSE), "recall": recall, "precision": precision,
                               "TP": len(predictions['TPImages']), "TN": len(predictions['TNImages']), 
                               "FP": len(predictions['FPImages']), "FN": len(predictions['FNImages'])})
            predictions.update(args)
            results.append(predictions)
        except Exception as e:
            print(path, e)        

112 114 0 2


In [5]:
df = pd.DataFrame.from_dict(results).drop(["TPImages", "TNImages", "FPImages", "FNImages",
                          "TPPredictions", "TNPredictions", "FPPredictions", "FNPredictions"], axis=1)
df.head(10)

,RMSE,recall,precision,TP,TN,FP,FN,feature,balance,batchsize,IW,IH,subset,trainDataTime,trainTime,timePerEpoch,testDataTime,predictTime
0,1.197737,0.982456,1.0,112,114,0,2,isGuiOpen,1,32,160,90,0.005,0.921871,259.807622,25.980762,9.185453,0.559052


In [6]:
df.describe()

,RMSE,recall,precision,TP,TN,FP,FN,balance,batchsize,IW,IH,subset,trainDataTime,trainTime,timePerEpoch,testDataTime,predictTime
count,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000,1.000000,1.000000,1.000000,1.000000,1.000000
mean,1.197737,0.982456,1.0,112.0,114.0,0.0,2.0,1.0,32.0,160.0,90.0,0.005,0.921871,259.807622,25.980762,9.185453,0.559052
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.197737,0.982456,1.0,112.0,114.0,0.0,2.0,1.0,32.0,160.0,90.0,0.005,0.921871,259.807622,25.980762,9.185453,0.559052
25%,1.197737,0.982456,1.0,112.0,114.0,0.0,2.0,1.0,32.0,160.0,90.0,0.005,0.921871,259.807622,25.980762,9.185453,0.559052
50%,1.197737,0.982456,1.0,112.0,114.0,0.0,2.0,1.0,32.0,160.0,90.0,0.005,0.921871,259.807622,25.980762,9.185453,0.559052
75%,1.197737,0.982456,1.0,112.0,114.0,0.0,2.0,1.0,32.0,160.0,90.0,0.005,0.921871,259.807622,25.980762,9.185453,0.559052
max,1.197737,0.982456,1.0,112.0,114.0,0.0,2.0,1.0,32.0,160.0,90.0,0.005,0.921871,259.807622,25.980762,9.185453,0.559052


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   RMSE           1 non-null      float64
 1   recall         1 non-null      float64
 2   precision      1 non-null      float64
 3   TP             1 non-null      int64  
 4   TN             1 non-null      int64  
 5   FP             1 non-null      int64  
 6   FN             1 non-null      int64  
 7   feature        1 non-null      object 
 8   balance        1 non-null      int64  
 9   batchsize      1 non-null      int64  
 10  IW             1 non-null      int64  
 11  IH             1 non-null      int64  
 12  subset         1 non-null      float64
 13  trainDataTime  1 non-null      float64
 14  trainTime      1 non-null      float64
 15  timePerEpoch   1 non-null      float64
 16  testDataTime   1 non-null      float64
 17  predictTime    1 non-null      float64
dtypes: float64(9),

In [10]:
feature = "isGuiOpen"
batchsize = 32
shrink = .25
IW, IH = (int(640*shrink), int(360*shrink))
subset = 0.005
balance = 1
sample = {}
for item in results:
    if item["feature"] == feature and item["batchsize"] == batchsize and item["IW"] == IW and item["IH"] == IH and item["subset"] == subset and item["balance"] == balance:
        sample = item
print("RMSE:", sample["RMSE"], "| Recall:", sample["recall"], "| Precision:", sample["precision"])
print("True Positives:", sample["TP"])
print("True Negatives:", sample["TN"])
print("False Positives:", sample["FP"])
print("False Negatives:", sample["FN"])
print("Train Time Total:", sample["trainTime"], "| AVG Time Per Epoch:", sample["timePerEpoch"], "| Epochs Ran:", (int(sample["trainTime"]/sample["timePerEpoch"])))



RMSE: 1.197736600924766 | Recall: 0.9824561403508771 | Precision: 1.0
True Positives: 112
True Negatives: 114
False Positives: 0
False Negatives: 2
Train Time Total: 259.80762219429016 | AVG Time Per Epoch: 25.980762219429018 | Epochs Ran: 10


In [11]:
for i in range(0, 5):
    index = randint(0, len(sample["TPImages"]))
    print(sample["TPImages"][index])
    print(sample["TPPredictions"][index])
    display(Image(filename=sample["TPImages"][index]))

../assets/datasets/BASALT Contractor Dataset/MineRLBasaltBuildVillageHouse-v0/Player347-1f668d2c7ba4-20220707-001339.mp4
0.9987944


ValueError: Cannot embed the 'mp4' image format

In [ ]:
resultPath = "../model_tests/Move Classifier/"


for path in os.listdir(resultPath):
    results = []
    if not '.DS_Store' in path:
        try:
            df = pd.read_csv(resultPath + path + "/predictions.csv")
            for index, row in df.iterrows():
                results.append({"image": row['image'].split("'")[3], 
                                "prediction": row['prediction'], 
                                "true": row['true'], 
                                "results": row['results']})
            df2 = pd.DataFrame.from_dict(results)
            df2.to_csv(resultPath + path + "/predictions2.csv", index=False)
        except:
            print(resultPath + path)
            